In [0]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

import math
import copy

from google.colab import drive
drive.mount('/content/drive')

class MLP(nn.Module):
  def __init__(self, n_units, init_scale=1.0):
    super(MLP, self).__init__()

    self._n_units = copy.copy(n_units)
    self._layers = []
    for i in range(1, len(n_units)):
      layer = nn.Linear(n_units[i-1], n_units[i], bias=False)
      variance = math.sqrt(2.0 / (n_units[i-1] + n_units[i]))
      layer.weight.data.normal_(0.0, init_scale * variance)
      self._layers.append(layer)

      name = 'fc%d' % i
      if i == len(n_units) - 1:
        name = 'fc'  # the prediction layer is just called fc
      self.add_module(name, layer)


  def forward(self, x):
    x = x.view(-1, self._n_units[0])
    out = self._layers[0](x)
    for layer in self._layers[1:]:
      out = F.relu(out)
      out = layer(out)
    return out


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd drive/'My Drive'

/content/drive/My Drive


In [0]:
def get_data(seed):
  normalize = transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]], std=[x / 255.0 for x in [63.0, 62.1, 66.7]])
  transform_train = transforms.Compose([
          transforms.ToTensor(),
          normalize])
  transform_test = transforms.Compose([
        transforms.ToTensor(),
        normalize])
  train_dataset = datasets.CIFAR10(root='data', 
                                train=True, 
                                transform=transform_train,
                                download=True)
  test_dataset = datasets.CIFAR10(root='data', 
                                train=False, 
                                transform=transform_test,
                                download=True)
  np.random.seed(seed)

  probability_of_random = 1.0
  labels = np.array(train_dataset.targets) 
  mask = np.random.rand(len(labels)) <= probability_of_random #create mask of length labels, where entries drawn from [0,1].
  rnd_labels = np.random.choice(10, mask.sum())               #create random labels 1-10 of length of mask
  labels[mask] = rnd_labels
  labels = [int(x) for x in labels]
  train_dataset.targets = labels                              #assign new random labels to dataset
  
  train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=128,
                          num_workers=4,
                          shuffle=True)
  test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=128,
                         num_workers=4,
                         shuffle=False)
  return train_loader, test_loader

In [0]:
def compute_acc(model, data_loader):
    correct_pred, num_examples = 0, 0
    model.eval()
    for i, (features, targets) in enumerate(data_loader):
        features, targets = features.to(device), targets.to(device)
        probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        assert predicted_labels.size() == targets.size()
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

In [47]:
criterion = nn.CrossEntropyLoss()

epoch_max = 200

for seed in range(10):
  torch.manual_seed(seed)
  mlp_spec = '512'
  n_units = [int(x) for x in mlp_spec.split('x')] # hidden dims
  n_units.append(10)  # output dim
  n_units.insert(0, 32*32*3)        # input dim
  model = MLP(n_units)
  optimizer = torch.optim.SGD(model.parameters(), .01, momentum=0.9, weight_decay=0)
  scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [50, 100, 150], gamma=0.1, last_epoch=-1)

  cost_list = []
  train_acc_list = []

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  model = model.to(device)

  train_loader, test_loader = get_data(seed)

  for epoch in range(epoch_max):
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
    # compute output
      features, targets = features.to(device), targets.to(device)
      output = model(features)
      loss = criterion(output, targets)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      cost_list.append(loss.item())
      if not batch_idx % 150:
        print (f'Epoch: {epoch+1:03d}/{epoch_max:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} |' 
                   f' Cost: {loss:.4f} |'
                   f'Seed {seed:03d} |' )
    if epoch % 10 == 0:
          torch.save(model.state_dict(), 'MLP_randomlabels_seed' + str(seed))
    model.eval()
    with torch.set_grad_enabled(False): # save memory during inference
        
        train_acc = compute_acc(model, train_loader)        
        print(f'Epoch: {epoch+1:03d}/{epoch_max:03d}\n'
              f'Train ACC: {train_acc:.2f}')
        train_acc = train_acc.cpu()
        train_acc_list.append(train_acc)
        zero = torch.Tensor([0.0])
        if torch.allclose(train_acc, zero):
          torch.save(model.state_dict(), 'MLP_randomlabels_seed' + str(seed))
    scheduler.step()

  np.save('cost_MLP_randomlabels_seed' + str(seed), cost_list)
  np.save('trainacc_MLP_randomlabels_seed' + str(seed), train_acc_list)

 

Streaming output truncated to the last 5000 lines.
Train ACC: 15.64
Epoch: 003/200 | Batch 000/391 | Cost: 2.2544 |Seed 005 |
Epoch: 003/200 | Batch 150/391 | Cost: 2.3162 |Seed 005 |
Epoch: 003/200 | Batch 300/391 | Cost: 2.3103 |Seed 005 |
Epoch: 003/200
Train ACC: 16.94
Epoch: 004/200 | Batch 000/391 | Cost: 2.2960 |Seed 005 |
Epoch: 004/200 | Batch 150/391 | Cost: 2.3157 |Seed 005 |
Epoch: 004/200 | Batch 300/391 | Cost: 2.3327 |Seed 005 |
Epoch: 004/200
Train ACC: 18.93
Epoch: 005/200 | Batch 000/391 | Cost: 2.2064 |Seed 005 |
Epoch: 005/200 | Batch 150/391 | Cost: 2.2155 |Seed 005 |
Epoch: 005/200 | Batch 300/391 | Cost: 2.2953 |Seed 005 |
Epoch: 005/200
Train ACC: 18.70
Epoch: 006/200 | Batch 000/391 | Cost: 2.2150 |Seed 005 |
Epoch: 006/200 | Batch 150/391 | Cost: 2.3323 |Seed 005 |
Epoch: 006/200 | Batch 300/391 | Cost: 2.2595 |Seed 005 |
Epoch: 006/200
Train ACC: 21.21
Epoch: 007/200 | Batch 000/391 | Cost: 2.1685 |Seed 005 |
Epoch: 007/200 | Batch 150/391 | Cost: 2.2166 |See